In [1]:
import pandas as pd
import tensorflow as tf
import os
from keras.utils import load_img
import numpy as np
import cv2

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import GlobalMaxPooling2D
tf.__version__

'2.12.0'

In [2]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg

In [3]:
DATASET_PATH = "C:/Programs/FashionRecommender/data/myntradataset/"
DATA_PATH = "C:/Programs/FashionRecommender/data/myntradataset/"
print(os.listdir(DATA_PATH))

['images', 'styles.csv']


In [4]:
df = pd.read_csv(DATA_PATH + "styles.csv", on_bad_lines="skip")
print(df.shape)
df.head()

(44424, 10)


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [5]:
%%time
import os

def get_all_filenames(directory):
    """
    Returns a set of all filenames in the given directory.
    """
    filenames = {entry.name for entry in os.scandir(directory) if entry.is_file()}
    return filenames

images = get_all_filenames(DATASET_PATH + "images/")

Wall time: 249 ms


In [6]:
def check_image_exists(image_filename):
    """
    Checks if the desired filename exists within the filenames found in the given directory.
    Returns True if the filename exists, False otherwise.
    """
    global images
    if image_filename in images:
        return image_filename
    else:
        return np.nan

df['image'] = df["id"].apply(lambda image: check_image_exists(str(image) + ".jpg"))
df = df.reset_index(drop=True)
df.head()
print(df.shape)

(44424, 11)


In [7]:
filtered_df = df[df['articleType'].isin(['Shirts', 'Trousers', 'Jeans','Track Pants', 'Tshirts', 'Socks', 
                                         'Tops', 'Sweatshirts', 'Kurtas', 'Waistcoat', 'Sarees', 'Rain Jacket', 
                                         'Dresses', 'Night suits', 'Skirts', 'Blazers', 'Kurta Sets', 'Shrug', 
                                         'Boxers', 'Dupatta', 'Bath Robe', 'Mufflers', 'Jackets', 'Trunk', 
                                         'Lounge Pants', 'Sweaters', 'Tracksuits', 'Swimwear', 'Nightdress', 
                                         'Leggings', 'Kurtis', 'Jumpsuit', 'Robe', 'Salwar and Dupatta', 
                                         'Patiala', 'Churidar', 'Lounge Tshirts', 'Lounge Shorts', 
                                         'Nehru Jackets', 'Salwar', 'Lehenga Choli', 'Clothing Set' ])]

In [9]:
df = filtered_df.dropna(subset=['image'])
print(df.shape)

(19390, 11)


In [10]:
def plot_figures(figures, nrows = 1, ncols=1,figsize=(8, 8)):
    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows,figsize=figsize)
    for ind,title in enumerate(figures):
        axeslist.ravel()[ind].imshow(cv2.cvtColor(figures[title], cv2.COLOR_BGR2RGB))
        axeslist.ravel()[ind].set_title(title)
        axeslist.ravel()[ind].set_axis_off()
    plt.tight_layout() # optional

def img_path(img,display=False):
    if display :
        return DATA_PATH+"/images/"+img
    else :
        return DATASET_PATH+"/images/"+img

def load_image(img,display=False):
    return cv2.imread(img_path(img,display))

In [12]:
# Input Shape
img_width, img_height, _ = load_image(df.iloc[0].image).shape

# Pre-Trained Model
base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape = (img_width, img_height, 3))
base_model.trainable = False

# Add Layer Embedding
model = tf.keras.Sequential([
    base_model,
    GlobalMaxPooling2D()
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 3, 2, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________
